In [11]:
import numpy as np
from tqdm import tqdm

In [12]:
def generate_PRN_code(prn):
    # Define the shift array for G2 code generation
    g2_shifts = [5, 6, 7, 8, 17, 18, 139, 140, 141, 251, 252, 254, 255, 256, 257, 258, 469, 470, 471, 472,
                 473, 474, 509, 512, 513, 514, 515, 516, 859, 860, 861, 862, 145, 175, 52, 21, 237, 235,
                 886, 657, 634, 762, 355, 1012, 176, 603, 130, 359, 595, 68, 386]

    # Determine the appropriate shift for the given PRN number for G2 code
    g2_shift = g2_shifts[prn - 1]

    # Generate G1 code sequence
    g1_code = np.ones(1023)
    g1_register = -np.ones(10)
    for i in range(1023):
        g1_code[i] = g1_register[9]
        feedback_bit = g1_register[2] * g1_register[9]
        g1_register[1:] = g1_register[:-1]
        g1_register[0] = feedback_bit

    # Generate G2 code sequence
    g2_code = np.ones(1023)
    g2_register = -np.ones(10)
    for i in range(1023):
        g2_code[i] = g2_register[9]
        feedback_bit = g2_register[1] * g2_register[2] * g2_register[5] * g2_register[7] * g2_register[8] * g2_register[9]
        g2_register[1:] = g2_register[:-1]
        g2_register[0] = feedback_bit

    # Shift the G2 code sequence
    g2_code = np.roll(g2_code, g2_shift)

    # Form the single sample C/A code by multiplying G1 and G2
    ca_code = -(g1_code * g2_code)

    return ca_code


In [13]:


# Define the path to the data file
data_path = 'C:/Users/visha/Documents/Jupyter notebook files/AE410 assignment/RTLSDR_Bands-L1.uint8'

# Load the data from the file
# The data is in 8-bit unsigned integer format
with open(data_path, 'rb') as file:
    raw_data = np.fromfile(file, dtype=np.uint8, count=4092)

# The I and Q components are interleaved in the file, so we need to separate them.
i_components = raw_data[0::2].astype(np.float64) - 128.0
q_components = raw_data[1::2].astype(np.float64) - 128.0

# Combine I and Q to form complex samples
iq_samples = i_components + 1j * q_components


In [14]:
len(iq_samples)

2046

In [15]:
def generate_ca_signal(ca_code, circular_shift, upsampling_factor, doppler_frequency):
    # Perform circular shift on the C/A code
    shifted_ca_code = np.roll(ca_code, circular_shift)
    
    # Upsample the code by repeating each element 'upsampling_factor' times
    upsampled_ca_code = np.repeat(shifted_ca_code, upsampling_factor)
    
    # Generate a time vector (assuming the code length is in milliseconds)
    time_vector = np.linspace(0, len(upsampled_ca_code) * 1e-3, len(upsampled_ca_code), endpoint=False)
    
    # Modulate the upsampled C/A code by a complex exponential to represent the Doppler shift
    doppler_shifted_ca = upsampled_ca_code * np.exp(1j * 2 * np.pi * doppler_frequency * time_vector)
    
    return doppler_shifted_ca


In [16]:


import numpy as np
from scipy.signal import correlate

def find_best_match(input_signal, doppler_range, num_prns, sample_prn_code, sampling_factor=2):
    best_match_info = {'doppler': 0, 'shift': 0, 'value': 0, 'prn': 0}
    max_correlation_value = 0

    for prn in range(1, num_prns + 1):
        prn_code = sample_prn_code(f'./CA_Codes/prn{prn}.txt')

        for shift in range(1023):  # 1023 shifts for PRN code
            for doppler in doppler_range:
                doppler_shifted_ca_code = generate_ca_signal(prn_code, shift, sampling_factor, doppler)
                correlation_result = correlate(input_signal, doppler_shifted_ca_code, mode='full')

                max_correlation = np.max(correlation_result)
                if max_correlation > max_correlation_value:
                    max_correlation_value = max_correlation
                    best_match_info.update({'doppler': doppler, 'shift': shift, 'prn': prn, 'value': max_correlation_value})

    return best_match_info

# The sample_prn_code and generate_ca_signal functions would need to be defined in Python as well.
# For example:
def sample_prn_code(file_path):
    # Read the PRN code from a file and return it as a numpy array
    with open(file_path, 'r') as file:
        return np.array([int(line.strip()) for line in file])



In [17]:
import numpy as np

def find_best_matches_1(input_signal, doppler_range, num_prns, sample_prn_code, sampling_factor=2):
    top_five_matches = []

    for prn in tqdm(range(1, num_prns + 1), desc='Loop 1'):
        prn_code = sample_prn_code(C:/Users/visha/Documents/Jupyter notebook files/AE410 assignment/CA_Codes/prn{prn}.txt')

        for shift in tqdm(range(1023), desc='Loop 2'):  # Assuming 1023 shifts for PRN code
            for doppler in doppler_range:
                doppler_shifted_ca_code = generate_ca_signal(prn_code, shift, sampling_factor, doppler)
                correlation_result = np.correlate(input_signal, doppler_shifted_ca_code, mode='full')

                max_correlation = np.max(correlation_result)
                if len(top_five_matches) < 5 or max_correlation > top_five_matches[0]['value']:
                    match_info = {'doppler': doppler, 'shift': shift, 'prn': prn, 'value': max_correlation}
                    if len(top_five_matches) >= 5:
                        top_five_matches[0] = match_info  # Replace the smallest
                    else:
                        top_five_matches.append(match_info)  # Append to the list
                    
                    # Sort the list based on the correlation value in descending order
                    top_five_matches.sort(key=lambda x: x['value'], reverse=True)

    return top_five_matches

# Make sure to define `generate_ca_signal` and `sample_prn_code` functions appropriately



In [21]:
print(find_best_matches_1(iq_samples, list(range(-5000,5000,500)),32,sample_prn_code= sample_prn_code))

Loop 1: 100%|██████████| 32/32 [25:35<00:00, 47.97s/it]

[{'doppler': -4000, 'shift': 386, 'prn': 13, 'value': (18236.00000001067+13659.999999958904j)}, {'doppler': -5000, 'shift': 0, 'prn': 1, 'value': (13615.00000001016+2893.0000000073633j)}, {'doppler': -3000, 'shift': 0, 'prn': 1, 'value': (13615.000000008495+2892.999999983588j)}, {'doppler': -3500, 'shift': 0, 'prn': 1, 'value': (8433.000000006261+7000.99999998222j)}, {'doppler': -4500, 'shift': 0, 'prn': 1, 'value': (8432.9999999961+7001.00000001241j)}]


In [22]:
from numpy.fft import fft, ifft

def parallel_code_phase_search(input_data, doppler_range, doppler_step, sampling_rate):
    s = sampling_rate
    best_match_info = {'doppler': 0, 'shift': 0, 'prn': 0}
    max_correlation_value = 0
    t = np.arange(0, 1, 1 / (s * 1023)) * 1e-3
    doppler_freq_list = np.arange(*doppler_range, doppler_step)

    for prn in tqdm(range(1, 33)):
        ca_code = generate_PRN_code(prn)
        ca_sampled = np.repeat(ca_code, s)
        ca_fft_sampled = fft(ca_sampled)

        for dp_freq in doppler_freq_list:
            sampling_freq = 2.048e6
            doppler_shift = np.exp(-1j * 2 * np.pi * dp_freq * np.arange(len(input_data)) / sampling_freq)

            input_shifted_fft = fft(input_data) * doppler_shift
            assert len(input_shifted_fft) == len(ca_fft_sampled), "The lengths of Ca_shifted and input_data should be the same."
            result_fft = input_shifted_fft * np.conj(ca_fft_sampled)

            correlation = abs(ifft(result_fft))

            if np.max(correlation) > max_correlation_value:
                max_correlation_value = np.max(correlation)
                best_match_info['doppler'] = dp_freq
                best_match_info['shift'] = np.argmax(correlation)
                best_match_info['prn'] = prn
        print(best_match_info)
    return best_match_info


In [23]:
parallel_code_phase_search(iq_samples, (-5000,5000), 500, 2)

 78%|███████▊  | 25/32 [00:00<00:00, 122.21it/s]

{'doppler': 4500, 'shift': 607, 'prn': 1}
{'doppler': -5000, 'shift': 1768, 'prn': 2}
{'doppler': 4500, 'shift': 1618, 'prn': 3}
{'doppler': 4500, 'shift': 1618, 'prn': 3}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4500, 'shift': 1462, 'prn': 5}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}


100%|██████████| 32/32 [00:00<00:00, 117.81it/s]

{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}
{'doppler': 4000, 'shift': 1400, 'prn': 13}


{'doppler': 4000, 'shift': 1400, 'prn': 13}